In [1]:
import pandas as pd
import numpy as np

In [2]:
result_path = '/home/sami/Claimspotting/Metadata/Results/Results.csv'

In [7]:
def format_for_final_results(df, metric):
    models = ['logreg', 'svm', 'XGBoost', 'xlm-roberta-base', 'ClaimbusterTransformer', 'GermevalEnsemble']
    trainsets = ['claimbuster', 'claimrank', 'checkthat2019', 'checkthat2021','checkthat2022', 'germeval']
    df = df[['Model', 'Train', 'Test', metric]]
    if metric == 'Precision':
        df = df[df['Test'] != 'multifc']
    dct_list = []
    for model in models:
        temp1 = df[df['Model'] == model].copy()
        for trainset in trainsets:
            temp2 = temp1[temp1['Train'] == trainset].copy()
            if temp2.shape[0] == 0:
                baseline = np.nan
                dct = {'Model':model, 'Train':trainset, 'Mean':np.nan, 'Std':np.nan}
            else:
                baseline = float(temp2[temp2['Test'] == trainset][metric])
            temp2['Difference'] = temp2[metric].apply(lambda score: score-baseline)
            temp2 = temp2[temp2['Test'] != trainset]
            results = temp2['Difference'].to_numpy().reshape(-1)
            mean = np.round(np.mean(results),2)
            std = np.round(np.std(results),2)
            dct = {'Model':model, 'Train':trainset, 'Mean':mean, 'Std':std}
            dct_list.append(dct)
    cols = {trainset:[] for trainset in trainsets}
    for trainset in trainsets:
        for model in models:
            for dct in dct_list:
                if dct['Model'] == model and dct['Train'] == trainset:
                    cols[trainset].append(f"{dct['Mean']}")
    final_frame = pd.DataFrame.from_dict(cols)
    data_appreviations = ['CB', 'CR', 'CT19', 'CT21', 'CT22', 'GE']
    final_frame = final_frame.rename(columns={old:new for old, new in zip(trainsets, data_appreviations)})
    final_frame = final_frame.astype(float)
    final_frame = final_frame.append(final_frame.astype(float).mean(axis = 0, skipna = True), ignore_index =True)
    final_frame['Average'] = final_frame.mean(axis = 1, skipna = True)
    final_frame = final_frame.rename(index={4: "Adv.Transformer", 1: "SVM", 0: "LogReg", 5:"Ensemble", 3:"Transformer", 2:"XGBoost", 6:"Average"})
    return final_frame.round(2)

def merge(recall_frame, precision_frame):
    dct = {col:[] for col in precision_frame.columns}
    for col in precision_frame.columns:
        for row in precision_frame.index:
            value = f"{recall_frame.loc[row,col]}/{precision_frame.loc[row,col]}"
            if value == "nan/nan":
                value = "NaN"
            dct[col].append(value)
    final = pd.DataFrame.from_dict(dct)
    final = final.rename(index={4: "Adv.Transformer", 1: "SVM", 0: "LogReg", 5:"Ensemble", 3:"Transformer", 2:"XGBoost", 6:"Average"})
    return final

def to_latex(df):
    print(df.to_latex(index=True, 
                header=True,
                bold_rows=True, 
                label='results'))  


In [8]:
import warnings

with warnings.catch_warnings(record=True):
    df = pd.read_csv(result_path)
    recall_frame = format_for_final_results(df,'Recall')
    df = pd.read_csv(result_path)
    precision_frame = format_for_final_results(df,'Precision')
    final = merge(recall_frame, precision_frame)
    display(final)

,CB,CR,CT19,CT21,CT22,GE,Average
LogReg,-0.28/-0.49,-0.34/-0.28,0.01/0.26,0.03/0.42,-0.24/-0.32,NaN,-0.16/-0.08
SVM,-0.1/-0.57,-0.09/-0.34,0.06/0.32,0.04/0.47,-0.18/-0.31,NaN,-0.05/-0.09
XGBoost,-0.33/-0.56,-0.44/-0.43,0.02/0.43,0.07/0.66,-0.12/-0.32,-0.03/-0.36,-0.14/-0.1
Transformer,-0.29/-0.59,-0.17/-0.19,0.07/0.34,0.04/0.4,0.02/-0.38,0.06/-0.38,-0.04/-0.13
Adv.Transformer,-0.28/-0.4,NaN,NaN,NaN,NaN,NaN,-0.28/-0.4
Ensemble,-0.22/-0.57,-0.43/-0.46,0.13/0.29,0.06/0.51,-0.15/-0.4,0.01/-0.4,-0.1/-0.17
Average,-0.25/-0.53,-0.29/-0.34,0.06/0.33,0.05/0.49,-0.13/-0.35,0.01/-0.38,-0.09/-0.13


In [5]:
to_latex(final)


\begin{table}
\centering
\label{results}
\begin{tabular}{llllllll}
\toprule
{} &           CB &           CR &       CT19 &       CT21 &         CT22 &           GE &      Average \\
\midrule
\textbf{LogReg         } &  -0.28/-0.49 &  -0.34/-0.28 &  0.01/0.26 &  0.03/0.42 &  -0.24/-0.32 &          NaN &  -0.16/-0.08 \\
\textbf{SVM            } &   -0.1/-0.57 &  -0.09/-0.34 &  0.06/0.32 &  0.04/0.47 &  -0.18/-0.31 &          NaN &  -0.05/-0.09 \\
\textbf{XGBoost        } &  -0.33/-0.56 &  -0.44/-0.43 &  0.02/0.43 &  0.07/0.66 &  -0.12/-0.32 &  -0.03/-0.36 &   -0.14/-0.1 \\
\textbf{Transformer    } &  -0.29/-0.59 &  -0.17/-0.19 &  0.07/0.34 &   0.04/0.4 &   0.02/-0.38 &   0.06/-0.38 &  -0.04/-0.13 \\
\textbf{Adv.Transformer} &   -0.28/-0.4 &          NaN &        NaN &        NaN &          NaN &          NaN &   -0.28/-0.4 \\
\textbf{Ensemble       } &  -0.22/-0.57 &  -0.43/-0.46 &  0.16/0.22 &  0.06/0.44 &  -0.15/-0.41 &    0.01/-0.4 &   -0.09/-0.2 \\
\textbf{Average        } &  -0.25/

/tmp/ipykernel_384789/1214237783.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=True,


In [25]:

def get_avg(df, train:str, test:str):
    if test == 'all':
        avg = df[df['Test'] == test][['Recall', 'Precision']].mean(axis = 0, skipna = True).round(2)
    elif train == 'all':
        avg = df[df['Train'] == train][['Recall', 'Precision']].mean(axis = 0, skipna = True).round(2)
    else:
        avg = df[(df['Train'] == train) & (df['Test'] == test)][['Recall', 'Precision']].mean(axis = 0, skipna = True).round(2)
    return avg

df = pd.read_csv(result_path)
train = 'germeval'
test = 'claimbuster'
get_avg(df, train, test)

Recall       0.63
Precision    0.53
dtype: float64

In [4]:
import plotly.express as px
import pandas as pd

In [5]:
def make_bold(value, max):
    if value == max:
        return "XYZtextbf{" + str(np.round(value,2)) + "}"
    else:
        return str(np.round(value,2))
def format(df, anchor:str):
    df = df[df['Train'] == anchor]
    test = ['claimbuster', 'claimrank', 'checkthat2019', 'checkthat2021',
       'checkthat2022', 'germeval', 'multifc']
    df_list = []
    for testset in test:
        temp = df[df['Test'] == testset][['Model', 'F1', 'Recall', 'Precision']].reset_index(drop=True)
        if anchor == 'germeval':
            temp['Model'] = [ 'Transformer Ensemble', 'Single Transformer', 'XGBoost']
            lst = [2, 1, 0]
            temp = temp.loc[lst]
        else:
            if testset != 'germeval':
                temp['Model'] = ['SVM', 'Logistic Regression ', 'Transformer Ensemble', 'Single Transformer', 'XGBoost']
                lst = [0, 1, 4, 3, 2]
                temp = temp.loc[lst]
            else:
                temp['Model'] = [ 'Transformer Ensemble', 'Single Transformer', 'XGBoost']
                lst = [2, 1, 0]
                temp = temp.loc[lst]
        for metric in ['F1', 'Recall', 'Precision']:
            if metric == 'Precision' and testset == 'multifc':
                temp[metric] = temp[metric].apply(lambda x: np.round(x,2))
                continue
            max = temp[metric].max()
            temp[metric] = temp[metric].apply(lambda x: make_bold(x, max))
        temp = temp.set_index('Model')
        display(temp)
        print(temp.to_latex(index=True, 
                label='results'))  

        
        df_list.append(temp)
    return pd.concat(df_list).set_index('Model')

In [6]:
df = pd.read_csv(result_path)
df.Train.unique()
frame = format(df, 'checkthat2022')
#display(frame)
#to_latex(frame)


,F1,Recall,Precision
Model,,,
SVM,0.11,0.06,0.36
Logistic Regression,0.27,0.27,0.27
XGBoost,0.24,0.19,0.32
Single Transformer,XYZtextbf{0.41},XYZtextbf{0.46},0.36
Transformer Ensemble,0.32,0.26,XYZtextbf{0.43}


\begin{table}
\centering
\label{results}
\begin{tabular}{llll}
\toprule
{} &               F1 &           Recall &        Precision \\
Model                &                  &                  &                  \\
\midrule
SVM                  &             0.11 &             0.06 &             0.36 \\
Logistic Regression  &             0.27 &             0.27 &             0.27 \\
XGBoost              &             0.24 &             0.19 &             0.32 \\
Single Transformer   &  XYZtextbf\{0.41\} &  XYZtextbf\{0.46\} &             0.36 \\
Transformer Ensemble &             0.32 &             0.26 &  XYZtextbf\{0.43\} \\
\bottomrule
\end{tabular}
\end{table}



/tmp/ipykernel_429420/3196467757.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(temp.to_latex(index=True,


,F1,Recall,Precision
Model,,,
SVM,0.09,0.05,0.25
Logistic Regression,0.22,0.25,0.19
XGBoost,0.19,0.19,0.19
Single Transformer,XYZtextbf{0.32},XYZtextbf{0.48},0.24
Transformer Ensemble,0.28,0.3,XYZtextbf{0.26}


\begin{table}
\centering
\label{results}
\begin{tabular}{llll}
\toprule
{} &               F1 &           Recall &        Precision \\
Model                &                  &                  &                  \\
\midrule
SVM                  &             0.09 &             0.05 &             0.25 \\
Logistic Regression  &             0.22 &             0.25 &             0.19 \\
XGBoost              &             0.19 &             0.19 &             0.19 \\
Single Transformer   &  XYZtextbf\{0.32\} &  XYZtextbf\{0.48\} &             0.24 \\
Transformer Ensemble &             0.28 &              0.3 &  XYZtextbf\{0.26\} \\
\bottomrule
\end{tabular}
\end{table}



...

/tmp/ipykernel_429420/3196467757.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(temp.to_latex(index=True,


,F1,Recall,Precision
Model,,,
SVM,0.09,0.04,1.0
Logistic Regression,0.41,0.26,1.0
XGBoost,0.34,0.2,1.0
Single Transformer,XYZtextbf{0.63},XYZtextbf{0.46},1.0
Transformer Ensemble,0.41,0.26,1.0


/tmp/ipykernel_429420/3196467757.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(temp.to_latex(index=True,


\begin{table}
\centering
\label{results}
\begin{tabular}{lllr}
\toprule
{} &               F1 &           Recall &  Precision \\
Model                &                  &                  &            \\
\midrule
SVM                  &             0.09 &             0.04 &        1.0 \\
Logistic Regression  &             0.41 &             0.26 &        1.0 \\
XGBoost              &             0.34 &              0.2 &        1.0 \\
Single Transformer   &  XYZtextbf\{0.63\} &  XYZtextbf\{0.46\} &        1.0 \\
Transformer Ensemble &             0.41 &             0.26 &        1.0 \\
\bottomrule
\end{tabular}
\end{table}



KeyError: "None of ['Model'] are in the columns"

In [69]:

frame

,F1,Recall,Precision
Model,,,
Logistic Regression,0.36,0.34,0.39
SVM,0.16,0.09,0.72
XGBoost,0.34,0.24,0.57
Single Transformer,\textbf{0.51},\textbf{0.4},0.72
Transformer Ensemble,0.48,0.36,\textbf{0.73}
Logistic Regression,0.66,0.74,0.59
SVM,0.32,0.2,\textbf{0.87}
XGBoost,0.74,0.67,0.82
Single Transformer,0.58,0.59,0.58


In [27]:
np.mean(np.array([.66, .43, .23, .14, .46, .7]))

0.4366666666666667

In [1]:
import pandas as pd

In [62]:
ensemble = 'GermevalEnsemble'
df = pd.read_csv('/home/sami/Claimspotting/Metadata/Emissions/emissions2.csv')
df[df['project_name']==ensemble].to_csv('/home/sami/Claimspotting/Metadata/Emissions/TransformerEnsembleEmissions.csv', index = False)

In [7]:
import plotly.express as px
path = '/home/sami/Claimspotting/Datasets/Blended_Testset.csv'
df = pd.read_csv(path)
df

,Text,Label,dataset,cluster,DATASET_ID
0,"Selena Gomez, Brad Pitt Getting “Cozy” And “Fl...",1,multifc,27,4
1,Thank you.,0,checkthat2019,60,6
2,"In Hillary Clinton's America, illegal immigran...",1,multifc,29,13
3,Says Obama doubled funding for the Pell Grant.,1,multifc,27,18
4,"That's why on day one, I will rip to shreds th...",0,checkthat2019,31,19
...,...,...,...,...,...
42700,We will win.,0,checkthat2021,19,142339
42701,U.S. comedian Kevin Hart in critical condition...,1,multifc,44,142341
42702,But they'll tell you his prescription wasn't g...,0,claimbuster,59,142345
42703,CNN host Fareed Zakaria posted a blog entry ca...,1,multifc,1,142346


In [39]:
fig = px.bar(df.groupby(['cluster','dataset']).size().reset_index().rename(columns={0:'count'}), 
        x="cluster", 
        y="count", 
        color="dataset", 
        color_discrete_sequence= px.colors.qualitative.Pastel, 
        template="plotly_white",
        orientation='v',title="Test set Distribution")
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout({'xaxis_title':'Cluster'})
fig.update_layout({'yaxis_title':'Amount of Sentences'})
fig.update_layout(legend=dict(title="Dataset"))
fig.show()

In [61]:
def plot_test_blend():
        visuals_path = '/home/sami/Claimspotting/Metadata/Visuals/'
        path = '/home/sami/Claimspotting/Datasets/Blended_Testset.csv'
        testset = pd.read_csv(path)
        color_scheme = ['rgb(246, 207, 113)',
                'rgb(248, 156, 116)',
                'rgb(220, 176, 242)',
                'rgb(135, 197, 95)',
                'rgb(158, 185, 243)',
                'rgb(254, 136, 177)',
                'rgb(201, 219, 116)',
                'rgb(139, 224, 164)',
                'rgb(180, 151, 231)',
                'rgb(102, 197, 204)',
                'rgb(179, 179, 179)']
        dct = {1:'checkworthy', 0:'not checkworthy'}
        df_label = testset.groupby(['cluster','Label']).size().reset_index().rename(columns={0:'count'})
        df_label['Label'] = df_label['Label'].apply(lambda x:dct[x])
        df_datasets = testset.groupby(['cluster','dataset']).size().reset_index().rename(columns={0:'count'})
        fig1 = px.bar(df_datasets, 
                x="cluster", 
                y="count", 
                color="dataset", 
                color_discrete_sequence= color_scheme, 
                template="plotly_white",
                orientation='v')
        fig1.update_layout({'xaxis_title':''})
        fig1.update_layout({'yaxis_title':'Amount of Sentences'})
        fig1.update_layout(legend=dict(title="Dataset"))
        fig2 = px.bar(df_label, 
                x="cluster", 
                y="count", 
                color="Label", 
                color_discrete_sequence= color_scheme, 
                template="plotly_white",
                orientation='v')
        fig2.update_layout(legend=dict(title="Label"))
        fig2.update_layout({'xaxis_title':'Cluster'})
        fig2.update_layout({'yaxis_title':'Amount of Sentences'})
        fig1.write_image(visuals_path + "PNG/BlendedTest_Datasets.png", scale=7)
        fig1.write_html(visuals_path + "HTML/BlendedTest_Datasets.html")
        fig2.write_image(visuals_path + "PNG/BlendedTest_Labels.png", scale=7)
        fig2.write_html(visuals_path + "HTML/BlendedTest_Labels.html")
        fig1.show()
        fig2.show()


plot_test_blend()

In [57]:
from plotly.subplots import make_subplots
color_scheme = ['rgb(246, 207, 113)',
                'rgb(248, 156, 116)',
                'rgb(220, 176, 242)',
                'rgb(135, 197, 95)',
                'rgb(158, 185, 243)',
                'rgb(254, 136, 177)',
                'rgb(201, 219, 116)',
                'rgb(139, 224, 164)',
                'rgb(180, 151, 231)',
                'rgb(102, 197, 204)',
                'rgb(179, 179, 179)']
dct = {1:'checkworthy', 0:'not checkworthy'}
df_label = df.groupby(['cluster','Label']).size().reset_index().rename(columns={0:'count'})
df_label['Label'] = df_label['Label'].apply(lambda x:dct[x])
df_datasets = df.groupby(['cluster','dataset']).size().reset_index().rename(columns={0:'count'})
fig = make_subplots(rows=2, cols=1)
fig1 = px.bar(df_datasets, 
        x="cluster", 
        y="count", 
        color="dataset", 
        color_discrete_sequence= color_scheme, 
        template="plotly_white",
        orientation='v')
fig1.update_layout({'xaxis_title':''})
fig1.update_layout({'yaxis_title':'Amount of Sentences'})
fig1.update_layout(legend=dict(title="Dataset"))
fig2 = px.bar(df_label, 
        x="cluster", 
        y="count", 
        color="Label", 
        color_discrete_sequence= color_scheme, 
        template="plotly_white",
        orientation='v')
fig2.update_layout(legend=dict(title="Label"))
fig2.update_layout({'xaxis_title':'Cluster'})
fig2.update_layout({'yaxis_title':'Amount of Sentences'})
fig1.show()
fig2.show()


In [49]:
dct = {1:'checkworthy', 0:'not checkworthy'}
df_label = df.groupby(['cluster','Label']).size().reset_index().rename(columns={0:'count'})
df_label['Label'] = df_label['Label'].apply(lambda x:dct[x])
df_count

,cluster,Label,count
0,0,not checkworthy,839
1,0,checkworthy,161
2,1,not checkworthy,409
3,1,checkworthy,201
4,2,not checkworthy,419
...,...,...,...
135,67,checkworthy,14
136,68,not checkworthy,250
137,68,checkworthy,72
138,69,not checkworthy,9


In [43]:
df

,Text,Label,dataset,cluster,DATASET_ID
0,"Selena Gomez, Brad Pitt Getting “Cozy” And “Fl...",1,multifc,27,4
1,Thank you.,0,checkthat2019,60,6
2,"In Hillary Clinton's America, illegal immigran...",1,multifc,29,13
3,Says Obama doubled funding for the Pell Grant.,1,multifc,27,18
4,"That's why on day one, I will rip to shreds th...",0,checkthat2019,31,19
...,...,...,...,...,...
42700,We will win.,0,checkthat2021,19,142339
42701,U.S. comedian Kevin Hart in critical condition...,1,multifc,44,142341
42702,But they'll tell you his prescription wasn't g...,0,claimbuster,59,142345
42703,CNN host Fareed Zakaria posted a blog entry ca...,1,multifc,1,142346
